In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

sns.set_style('whitegrid')

C:\Users\afm6343\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
C:\Users\afm6343\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
avg = 1
std_dev = .1
num_reps = 500
num_simulations = 1000

In [3]:
#numpy to generate a list of percentages that will replicate our historical normal distribution: 
#round to 2 decimal places
pct_to_target = np.random.normal(avg, std_dev, num_reps).round(2)

In [4]:
print(pct_to_target)

[0.99 1.13 1.12 0.99 1.05 0.67 0.91 1.11 1.06 1.02 1.12 1.06 0.92 1.12
 1.07 0.95 1.1  0.98 1.04 0.89 1.03 1.1  1.01 0.88 1.23 0.88 1.14 1.15
 0.96 0.9  0.81 1.09 0.95 1.18 1.04 0.97 0.97 1.19 1.05 0.98 1.03 0.9
 0.93 0.78 1.23 0.95 0.82 1.13 1.18 1.03 1.02 1.08 0.88 1.02 0.84 1.
 1.11 0.97 1.22 1.2  1.   0.83 1.05 1.13 0.97 1.08 1.15 0.99 1.   0.99
 0.89 1.01 1.07 0.93 0.98 1.02 1.18 1.03 0.94 1.07 1.11 0.8  1.03 1.05
 1.07 0.87 0.96 1.15 0.99 0.99 0.91 0.96 0.79 0.96 0.95 0.89 0.94 0.96
 0.94 1.06 1.14 1.03 0.97 1.02 0.94 1.03 0.96 1.12 1.17 1.03 0.98 1.16
 0.95 1.02 1.01 0.98 0.94 1.09 1.1  1.01 1.19 0.96 0.9  0.96 1.02 0.86
 1.09 1.08 1.01 1.07 0.93 1.05 0.92 1.06 0.97 0.96 1.04 0.9  1.   1.02
 1.   0.99 0.97 1.24 0.98 1.02 0.96 1.06 1.02 1.08 0.87 1.05 0.84 1.01
 0.91 1.11 0.78 1.16 1.07 0.98 0.92 0.89 0.98 0.96 0.92 1.   1.13 1.07
 1.02 0.86 0.95 1.05 0.99 0.89 1.1  1.03 1.07 1.14 1.05 0.91 0.89 0.95
 0.98 0.86 0.84 0.89 1.01 1.02 0.97 0.92 0.92 1.03 1.01 0.97 0.84 1.03
 0.9  0.8

In [5]:
sales_target_values = [75_000, 100_000, 200_000, 300_000, 400_000, 500_000]
sales_target_prob = [.3, .3, .2, .1, .05, .05]
sales_target = np.random.choice(sales_target_values, num_reps, p=sales_target_prob)

In [6]:
df = pd.DataFrame(index=range(num_reps), data={'Pct_To_Target': pct_to_target,
                                               'Sales_Target': sales_target})

df['Sales'] = df['Pct_To_Target'] * df['Sales_Target']

In [8]:
df.head()

,Pct_To_Target,Sales_Target,Sales
0,0.99,200000,198000.0
1,1.13,200000,226000.0
2,1.12,75000,84000.0
3,0.99,300000,297000.0
4,1.05,100000,105000.0


In [9]:
def calc_commission_rate(x):
    """ Return the commission rate based on the table:
    0-90% = 2%
    91-99% = 3%
    >= 100 = 4%
    """
    if x <= .90:
        return .02
    if x <= .99:
        return .03
    else:
        return .04

In [10]:
df['Commission_Rate'] = df['Pct_To_Target'].apply(calc_commission_rate)
df['Commission_Amount'] = df['Commission_Rate'] * df['Sales']

In [11]:
df.head()

,Pct_To_Target,Sales_Target,Sales,Commission_Rate,Commission_Amount
0,0.99,200000,198000.0,0.03,5940.0
1,1.13,200000,226000.0,0.04,9040.0
2,1.12,75000,84000.0,0.04,3360.0
3,0.99,300000,297000.0,0.03,8910.0
4,1.05,100000,105000.0,0.04,4200.0


In [16]:
df.Commission_Amount.sum()

2888345.0

In [17]:
# Define a list to keep all the results from each simulation that we want to analyze
all_stats = []

# Loop through many simulations
for i in range(num_simulations):

    # Choose random inputs for the sales targets and percent to target
    sales_target = np.random.choice(sales_target_values, num_reps, p=sales_target_prob)
    pct_to_target = np.random.normal(avg, std_dev, num_reps).round(2)

    # Build the dataframe based on the inputs and number of reps
    df = pd.DataFrame(index=range(num_reps), data={'Pct_To_Target': pct_to_target,
                                                   'Sales_Target': sales_target})

    # Back into the sales number using the percent to target rate
    df['Sales'] = df['Pct_To_Target'] * df['Sales_Target']

    # Determine the commissions rate and calculate it
    df['Commission_Rate'] = df['Pct_To_Target'].apply(calc_commission_rate)
    df['Commission_Amount'] = df['Commission_Rate'] * df['Sales']

    # We want to track sales,commission amounts and sales targets over all the simulations
    all_stats.append([df['Sales'].sum().round(0),
                      df['Commission_Amount'].sum().round(0),
                      df['Sales_Target'].sum().round(0)])

In [18]:
results_df = pd.DataFrame.from_records(all_stats, columns=['Sales',
                                                           'Commission_Amount',
                                                           'Sales_Target'])

In [19]:
results_df.describe().style.format('{:,}')

,Sales,Commission_Amount,Sales_Target
count,"1,000.0","1,000.0","1,000.0"
mean,"83,678,666.0","2,858,013.436","83,665,725.0"
std,"2,879,197.596378719","108,600.07451145604","2,815,401.5851136516"
min,"74,786,000.0","2,507,095.0","75,350,000.0"
25%,"81,788,187.5","2,786,349.0","81,750,000.0"
50%,"83,785,250.0","2,856,716.5","83,837,500.0"
75%,"85,624,625.0","2,928,950.5","85,575,000.0"
max,"92,942,250.0","3,208,122.0","92,575,000.0"
